## Algoritmo Exaustivo

In [1]:
def proximo(offset, num_seqs, limite):
    pos = 0

    while pos < num_seqs:
        offset[pos] +=1

        if offset[pos] < limite:
            return offset

        offset[pos] = 0
        pos += 1

def enumerar(num_seqs,tam_seqs,tam_motifs,):
    estado = [0] * num_seqs
    limite = tam_seqs - tam_motifs + 1

    while estado is not None:
        yield tuple(estado)
        estado = proximo(estado, num_seqs, limite)

def score(seqs, offset, tam_motif):
    snips = [s[p: p + tam_motif] for p,s in zip(offset, seqs)]
    return sum(max([col.count(x) for x in col]) for col in zip(*snips))

def motif(seqs, num_seqs, tam_seq, tam_motif):

    assert all(len(s) == tam_seq for s in seqs)
    
    maior_s = 0
    for estado in enumerar(num_seqs, tam_seq, tam_motif):
        atual = score(seqs, estado, tam_motif)
        if atual > maior_s:
            melhor_pos = estado
            maior_s = atual

    return (melhor_pos, maior_s)

In [2]:
seqs = "ATGGTCGC TTGTCTGA CCGTAGTA".split()
motif(seqs, 3, 8, 3)

((3, 2, 2), 8)

In [3]:
seqs = "ATGGTCGC TTGTCTGA CCGTAGTA ATGCTAGC ATGGGTAG AGTAGCGC GGTAGATG TATATAAG".split()
motif(seqs, 8, 8, 3)

((1, 0, 3, 4, 5, 2, 2, 0), 21)

In [9]:
seqs = """cctgatagacgctatctggctatccacgtacgtaggtcctctgtgcgaatctatgcgtttccaaccat
agtactggtgtacatttgatacgtacgtacaccggcaacctgaaacaaacgctcagaaccagaagtgc
aaacgtacgtgcaccctctttcttcgtggctctggccaacgagggctgatgtataagacgaaaatttt
agcctccgatgtaagtcatagctgtaactattacctgccacccctattacatcttacgtacgtataca
ctgttatacaacgcgtcatggcggggtatgcgttttggtcgtcgtacgctcgatcgttaacgtacgtc""".splitlines()
motif(seqs, 5, len(seqs[0]), 8)

KeyboardInterrupt: 

# Branch and Bound - Método de Busca Otimizada

## Projeto de Alto Nível

1. **Entrada e Saída**

    **Entrada**: Lista de sequências de DNA e comprimento do motif a ser encontrado.

    **Saída**: Posições dos melhores motifs identificados e a pontuação do alinhamento.

2. **Componentes Principais**

    **Inicialização**: Define variáveis de estado e inicia a busca com offsets zerados.

    **Geração de Ramos**: Explora todas as possibilidades de alinhamento de motifs dentro dos limites da sequência.

    **Avaliação Parcial**: Calcula a pontuação dos motifs formados até um certo nível da busca.

    **Bound**: Descarta ramos cujas pontuações parciais sejam inferiores ao melhor resultado encontrado até o momento.

    **Critério de Paragem**: A busca termina quando todos os offsets forem preenchidos ou quando um ramo for eliminado pela poda.

3. **Fluxo de Dados**: Entrada das sequências e comprimento do motif → Inicialização das variáveis → Exploração iterativa das possibilidades → Cálculo da pontuação parcial e aplicação da poda → Retorno das melhores posições e pontuação final.

## Projeto de Baixo Nível

1. **Cálculo da Pontuação (score_bb)**

    **Descrição**: Determina a qualidade do alinhamento atual, contando os nucleotídeos mais frequentes em cada posição do motif.

    **Algoritmo**:

        Extrai os segmentos de DNA correspondentes aos offsets escolhidos.

        Para cada coluna do alinhamento, conta a ocorrência de cada nucleotídeo.

        A pontuação é a soma das frequências do nucleotídeo mais comum em cada coluna.

2. **Exploração Branch and Bound (branch_and_bound)**

    **Descrição**: Implementa a estratégia de busca Branch and Bound, iterativamente expandindo os offsets e eliminando soluções subótimas.

    **Algoritmo**:

        Se todos os offsets foram definidos, calcula a pontuação final.

        Se a pontuação atual for maior que a melhor encontrada, atualiza o estado global.

        Para cada posição possível na sequência:
        
            Define o offset correspondente.

            Calcula a pontuação parcial para prever a melhor pontuação possível.

        Se a previsão não ultrapassa a melhor pontuação encontrada, descarta o ramo.

        Caso contrário, continua a busca recursivamente.

3. **Execução do Algoritmo Principal (motif_bb)**

    **Descrição**: Função de alto nível que inicializa o estado global e chama o algoritmo de Branch and Bound.

    **Algoritmo**:

        Verifica se todas as sequências possuem o mesmo comprimento.

        Inicializa as variáveis de estado.

        Chama a função branch_and_bound para explorar as soluções.

        Retorna as melhores posições de motifs e sua pontuação.

4. **Saída e Visualização**

    Exibe a melhor configuração de posições encontradas.

    Mostra os segmentos extraídos das sequências que representam os motifs identificados.

    Retorna a pontuação do melhor alinhamento obtido.

In [ ]:
def score_bb(seqs, offset, tam_motif):
    # Extract motifs from sequences based on the given offsets
    snips = [s[p: p + tam_motif] for p, s in zip(offset, seqs)]
    
    # Calculate the score: sum of the highest frequency of any nucleotide/character in each column
    return sum(max(col.count(x) for x in set(col)) for col in zip(*snips))

def branch_and_bound(offset, num_seqs, limite, tam_motifs, estado_global, seqs, level=0):
    """ Branch and Bound implementation to find the best motif alignment """
    
    # Base case: if all offsets have been filled
    if level == num_seqs:  
        # Calculate the score for the current motif alignment
        atual = score_bb(seqs, offset, tam_motifs)
        
        # If the current score is better than the global best, update the best score and positions
        if atual > estado_global["maior_s"]:
            estado_global["maior_s"] = atual
            estado_global["melhor_pos"] = offset[:]
        else:
            return

    # Branching: Generate candidates for the current level
    for pos in range(limite):
        offset[level] = pos  # Set the current offset for this sequence
        
        # Calculate the partial score with the new offset configuration
        atual = score_bb(seqs, offset[:level+1], tam_motifs)

        # Bounding: If the maximum possible score for this branch is still better than the global best, explore it
        limite_superior = atual + (num_seqs - level - 1) * tam_motifs
        
        # If the upper bound of this branch is greater than the current best score, recurse further
        if limite_superior > estado_global["maior_s"]:
            branch_and_bound(offset, num_seqs, limite, tam_motifs, estado_global, seqs, level+1)

def motif_bb(seqs, num_seqs, tam_seq, tam_motif):
    """ Function to start the Branch and Bound search for the best motif alignment """
    
    # Ensure all sequences are of the expected length
    assert all(len(s) == tam_seq for s in seqs)

    # Initialize the global state: store the best score and positions
    estado_global = {"maior_s": 0, "melhor_pos": None}
    
    # Initialize the list of offsets (start at position 0 for each sequence)
    offset = [0] * num_seqs
    
    # Calculate the limit for valid offset values (based on sequence length and motif length)
    limite = tam_seq - tam_motif + 1

    # Start the Branch and Bound process
    branch_and_bound(offset, num_seqs, limite, tam_motif, estado_global, seqs)

    # Return the best motif positions and the corresponding score found
    return estado_global["melhor_pos"], estado_global["maior_s"]

In [ ]:
seqs = "ATGGTCGC TTGTCTGA CCGTAGTA".split()
motif_bb(seqs, 3, 8, 3)

([3, 2, 2], 8)

In [8]:
seqs = """cctgatagacgctatctggctatccacgtacgtaggtcctctgtgcgaatctatgcgtttccaaccat
agtactggtgtacatttgatacgtacgtacaccggcaacctgaaacaaacgctcagaaccagaagtgc
aaacgtacgtgcaccctctttcttcgtggctctggccaacgagggctgatgtataagacgaaaatttt
agcctccgatgtaagtcatagctgtaactattacctgccacccctattacatcttacgtacgtataca
ctgttatacaacgcgtcatggcggggtatgcgttttggtcgtcgtacgctcgatcgttaacgtacgtc""".splitlines()
motif_bb(seqs, 5, len(seqs[0]), 8)

([25, 20, 2, 55, 59], 40)

## Gibbs Sampling - Método Heurístico Estocástico

### Projeto de Alto Nível

1. **Entrada e Saída**:
   - **Entrada**: Lista de sequências de DNA e comprimento do motif a ser encontrado.
   - **Saída**: Posições dos melhores motifs identificados e a pontuação do alinhamento.

2. **Componentes Principais**:
   - **Inicialização Aleatória**: Seleciona aleatoriamente posições iniciais para os motifs em cada sequência.
   - **Cálculo da Matriz de Ponderação (PWM)**: Gera uma matriz de probabilidades baseada nas frequências de nucleótidos nos motifs selecionados.
   - **Ajuste Estocástico das Posições**: Utiliza um método de seleção por roleta para atualizar posições de motifs com base nas probabilidades calculadas.
   - **Critério de Paragem**: O algoritmo executa até atingir o número máximo de iterações, mantendo a melhor configuração encontrada com base na pontuação do alinhamento.

3. **Fluxo de Dados**:
   Entrada das sequências e comprimento do motif → Inicialização das posições → Atualização iterativa das posições usando PWM e seleção por roleta → Armazenamento da melhor solução → Retorno das melhores posições e pontuação.

---

### Projeto de Baixo Nível

1. **Inicialização Aleatória das Posições**:
   - **Descrição**: Gera posições iniciais aleatórias para os motifs dentro de cada sequência.
   - **Algoritmo**:
     - Para cada sequência, gera um índice aleatório dentro do intervalo permitido pelo comprimento do motif.
     - Armazena as posições num dicionário.

2. **Criação da PWM (Matriz de Ponderação)**:
   - **Descrição**: Constrói uma matriz de probabilidades de ocorrência dos nucleotídeos em cada posição do motif.
   - **Algoritmo**:
     - Extrai os motifs com base nas posições atuais.
     - Conta a frequência de cada nucleotídeo em cada posição.
     - Aplica pseudo contagem para evitar probabilidades nulas.
     - Converte as contagens em probabilidades.

3. **Cálculo das Probabilidades de motif**:
   - **Descrição**: Calcula a probabilidade de cada subsequência numa sequência ser um motif com base na PWM.
   - **Algoritmo**:
     - Percorre todas as subsequências possíveis dentro da sequência alvo.
     - Calcula a probabilidade de cada subsequência usando a PWM.
     - Normaliza os valores para obter uma distribuição de probabilidades.

4. **Seleção Estocástica da Nova Posição (Roleta)**:
   - **Descrição**: Escolhe uma nova posição para o motif na sequência alvo com base nas probabilidades calculadas.
   - **Algoritmo**:
     - Gera um número aleatório dentro do intervalo da soma acumulada das probabilidades.
     - Percorre a distribuição de probabilidades e seleciona a posição correspondente.

5. **Execução do Algoritmo Gibbs Sampling**:
   - **Descrição**: Iterativamente ajusta as posições dos motifs para maximizar a similaridade.
   - **Algoritmo**:
     - Inicializa posições aleatórias.
     - Em cada iteração:
       - Remove uma sequência da análise.
       - Calcula a PWM das demais sequências.
       - Avalia as probabilidades de motif na sequência removida.
       - Escolhe uma nova posição usando seleção por roleta.
       - Atualiza as posições e recalcula a pontuação.
       - Se a pontuação for melhor que a anterior, atualiza o melhor alinhamento e zera o contador de iterações sem melhoria.
       - Caso contrário, incrementa o contador.
     - O algoritmo executa até atingir o número máximo de iterações, mas armazena a melhor configuração encontrada ao longo da execução.

6. **Saída e Visualização**:
   - **Descrição**: Exibe as posições finais dos motifs encontrados e os segmentos extraídos das sequências.
   - **Algoritmo**:
     - Exibe a melhor configuração de posições.
     - Mostra os segmentos das sequências identificados como motifs.

In [21]:
import random

class GibbsSampling:
    def __init__(self, sequences, motif_length, pseudo=1):
        '''
        Inicializa a classe GibbsSampling.
        
        Parâmetros:
            sequences (list): Lista de sequências de DNA.
            motif_length (int): Tamanho do motif a ser descoberto.
            pseudo (float): Pequena constante para evitar probabilidades zero na PWM (padrão=1).
        '''
        self.seqs = sequences
        self.w = motif_length
        self.pseudo = pseudo
        self.n = len(sequences)
        self.t = len(sequences[0])

    def random_init_positions(self):
        '''
        Seleciona posições iniciais aleatórias para os motifs em cada sequência.
        
        Retorna:
            dict: Dicionário com sequências como chaves e posições iniciais como valores.
        '''
        return {seq: random.randint(0, self.t - self.w) for seq in self.seqs}

    def create_motifs(self, positions):
        '''
        Extrai motifs das sequências com base nas posições fornecidas.
        
        Parâmetros:
            positions (dict): Dicionário com posições iniciais para cada sequência.
        
        Retorna:
            list: Lista de motifs extraídos.
        '''
        return [seq[positions[seq]:positions[seq] + self.w] for seq in positions]

    def pwm(self, motifs):
        '''
        Calcula a Matriz de Pesos de Posição (PWM) a partir de um conjunto de motifs.
        
        Parâmetros:
            motifs (list): Lista de motifs.
        
        Retorna:
            list: PWM representada como uma lista de dicionários.
        '''
        bases = 'ATCG'
        pwm_matrix = []
        for pos in zip(*motifs):
            counts = {base: pos.count(base) + self.pseudo for base in bases}
            total = sum(counts.values())
            pwm_matrix.append({base: counts[base] / total for base in bases})
        return pwm_matrix

    def prob_seq(self, seq, pwm):
        '''
        Calcula a probabilidade de uma sequência dada a PWM.
        
        Parâmetros:
            seq (str): Sequência de DNA.
            pwm (list): Matriz de Pesos de Posição.
        
        Retorna:
            float: Probabilidade da sequência dada a PWM.
        '''
        prob = 1.0
        for i, base in enumerate(seq):
            prob *= pwm[i][base]
        return prob

    def prob_positions(self, seq, pwm):
        '''
        Calcula as probabilidades de cada posição da sequência conter o motif.
        
        Parâmetros:
            seq (str): Sequência de DNA.
            pwm (list): Matriz de Pesos de Posição.
        
        Retorna:
            list: Lista de probabilidades para cada posição possível do motif.
        '''
        probabilities = [self.prob_seq(seq[i:i+self.w], pwm) for i in range(self.t - self.w + 1)]
        total = sum(probabilities)
        return [p / total for p in probabilities]

    def roulette_wheel(self, probabilities):
        '''
        Realiza uma seleção estocástica usando o método da roleta.
        
        Parâmetros:
            probabilities (list): Lista de probabilidades.
        
        Retorna:
            int: Índice selecionado com base nas probabilidades.
        '''
        r = random.uniform(0, sum(probabilities))
        s = 0
        for i, p in enumerate(probabilities):
            s += p
            if s >= r:
                return i
        return len(probabilities) - 1

    def gibbs_sampling(self, max_iter=100, threshold=50):
        '''
        Executa o algoritmo de Amostragem de Gibbs para encontrar motifs.
        
        Parâmetros:
            max_iter (int): Número máximo de iterações (padrão=100).
            threshold (int): Critério de parada baseado na ausência de melhoria (padrão=50).
        
        Retorna:
            tuple: Melhores posições dos motifs e maior pontuação encontrada.
        '''
        positions = self.random_init_positions()
        best_positions = positions.copy()
        best_score = 0
        count = 0

        for _ in range(max_iter):
            if count < threshold:
                seq_to_remove = random.choice(self.seqs)
                temp_positions = positions.copy()
                temp_positions.pop(seq_to_remove)
                motifs = self.create_motifs(temp_positions)
                pwm = self.pwm(motifs)
                probabilities = self.prob_positions(seq_to_remove, pwm)
                new_pos = self.roulette_wheel(probabilities)
                positions[seq_to_remove] = new_pos

                current_motifs = self.create_motifs(positions)
                pwm_all = self.pwm(current_motifs)
                score = sum(max(col.values()) for col in pwm_all)

                if score > best_score:
                    best_score = score
                    best_positions = positions.copy()
                    count = 0
                else:
                    count += 1

        return best_positions, best_score

    def print_motif(self, positions):
        '''
        Imprime o alinhamento do motif descoberto.
        
        Parâmetros:
            positions (dict): Dicionário das melhores posições dos motifs.
        '''
        for seq in self.seqs:
            if seq in positions:
                start = positions[seq]
                print(seq[start:start+self.w])


def motif_gibbs(seqs, num_seqs, tam_seq, tam_motif, max_iter=100, threshold=50):
    assert len(seqs) == num_seqs
    assert all(len(s) == tam_seq for s in seqs)

    gibbs = GibbsSampling(seqs, tam_motif)
    positions, score = gibbs.gibbs_sampling(max_iter=max_iter, threshold=threshold)
    return positions, score


In [29]:
seqs = "ATGGTCGC TTGTCTGA CCGTAGTA".split()
motif_gibbs(seqs, 3, 8, 3)


({'ATGGTCGC': 2, 'TTGTCTGA': 1, 'CCGTAGTA': 1}, 1.4285714285714284)

Code for PWM

### Unit testing

função a gerar pedaço de dna

fazer teste à mão

injetar sequencia do teste à mão aos pedaços de dna

opcional: mudar nulceotido da sequencia ingetada para não ser biased